# Combine geojson files for current locations

In [1]:
import geojson

## New York City

In [2]:
with open('new_york.geojson') as f:
    nyc_data = geojson.load(f)

In [3]:
print(len(nyc_data.features))

3


In [4]:
nyc_data.features[0].keys()

dict_keys(['type', 'id', 'geometry', 'properties'])

In [5]:
nyc_data.features[0].properties

{'@id': 'http://nyc.pediacities.com/New_York_City', 'city': 'New York City'}

In [6]:
nyc_feature = geojson.Feature(
    geometry=geojson.MultiPolygon([p.geometry for p in nyc_data.features]),
    properties={'name': 'New York City', 'score': 50},
    id="3651000"
)
new_nyc_data = geojson.FeatureCollection([nyc_feature])

## Counties

- https://github.com/python-visualization/folium/blob/master/tests/us-counties.json
- http://eric.clst.org/tech/usgeojson/

In [7]:
with open('gz_2010_us_050_00_500k.json', encoding='latin1') as f:
    data = geojson.load(f)

In [8]:
data.features[0].properties['GEO_ID']

'0500000US01029'

In [9]:
FIPS_TO_SELECT = ["0500000US11001", "0500000US24031", "0500000US51059"]

In [10]:
counties_features = []
for feature in data.features:
    if feature.properties['GEO_ID'] in FIPS_TO_SELECT:
        print(feature.properties['GEO_ID'])
        feature.properties.update({'score': 85})
        counties_features.append(feature)
counties_data = geojson.FeatureCollection(counties_features)

0500000US11001
0500000US24031
0500000US51059


## Save new data

In [11]:
ADDITIONAL_PROPERTIES = {
    "Montgomery": {"slug": "Montgomery_County_MD", "score": 50, "spending_per_capita": 5359},
    "District of Columbia": {"slug": "Washington_DC", "score": 85, "spending_per_capita": 18271},
    "New York City": {"slug": "New_York_NY", "score": 30, "spending_per_capita": 10333},
    "Fairfax": {"slug": "Fairfax_County_VA", "score": 65, "spending_per_capita": 3625},
}

In [12]:
combined_data = geojson.FeatureCollection([nyc_feature] + counties_features)

In [13]:
for feature in combined_data.features:
    name = feature.properties.get('NAME', '') or feature.properties.get('name', '')
    if name in ADDITIONAL_PROPERTIES:
        feature.properties.update(ADDITIONAL_PROPERTIES[name])
        feature.properties['name'] = name

In [14]:
with open('locations.geojson', 'w') as f:
    geojson.dump(combined_data, f, indent=2)